# `1. Data Preparation`

In [20]:
# import all necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime

In [21]:
# import the data sets
cologneData = pd.read_csv("../../Data sets/koeln.csv")
essenData = pd.read_csv("../../Data sets/essen.csv")

In [22]:
# clean cologne data set of trips outside of cologne
cologneData = cologneData[cologneData['orig_lat'] >= 50]
cologneData = cologneData[cologneData['orig_lat'] < 52]
cologneData = cologneData[cologneData['orig_lng'] >= 6]
cologneData = cologneData[cologneData['orig_lng'] < 8]

cologneData = cologneData[cologneData['orig_lat'] >= 50]
cologneData = cologneData[ cologneData['orig_lat'] < 52]
cologneData = cologneData[cologneData['orig_lng'] >= 6]
cologneData = cologneData[ cologneData['orig_lng'] < 8]

essenData = essenData[essenData['orig_lat'] >= 50]
essenData = essenData[ essenData['orig_lat'] < 52]
essenData = essenData[essenData['orig_lng'] >= 6]
essenData = essenData[ essenData['orig_lng'] < 8]

**The following code block adds the columns to the rental bike dataset:**  

"numOfRentedBikes": shows the number of rented bikes for each hour  
"Zeitstempel": adds a datetimeobject which is hh:00:00, so it can be compared to the following weather data  

**to the Dataframe, and deletes all old columns**

In [23]:
#The weekday() function of date class in datetime module, returns an integer corresponding to the day of the week.  
def weekday_match (ts):
    return ts.weekday()
#returns the hour
def hour_match (ts):
    return ts.hour

#creates a datetime object from a normal object
def createDatetime (ts):
    return datetime.strptime(str(ts), '%Y-%m-%d').date()

#creates a datetime object from an object
def createTime (ts):
    return datetime.strptime(ts, '%H:%M:%S').time()

# creates a timestamp that is comparable to weather data
def timestamp_create(stamp):
    newStamp =  datetime.strptime(stamp, '%Y-%m-%d %H:%M:%S')   
    newStamp = newStamp.replace(minute=0, second=0)   
    return newStamp

# concatenate day and time strings as preparation for timestamp creation
essenData["Zeitstempel"] = essenData["day"] + ' ' + essenData["time"]
cologneData["Zeitstempel"] = cologneData["day"] + ' ' + cologneData["time"]
# create timestamp for comparison of bike rental data and weather data
essenData["Zeitstempel"] = essenData["Zeitstempel"].map(timestamp_create)
cologneData["Zeitstempel"] = cologneData["Zeitstempel"].map(timestamp_create)
# change datatypes to make them more comparable and accessable
essenData["day"] = essenData["day"].map(createDatetime)
essenData["time"] = essenData["time"].map(createTime)
cologneData["day"] = cologneData["day"].map(createDatetime)
cologneData["time"] = cologneData["time"].map(createTime)

# add weekday and hour feature
# essenData["weekday"]=essenData["day"].map(weekday_match)
# essenData["hour"] = essenData["time"].map(hour_match)
# cologneData["weekday"]=cologneData["day"].map(weekday_match)
# cologneData["hour"] = cologneData["time"].map(hour_match)

In [24]:
#delete ALL OLD COLUMNS
cologneData = cologneData.drop(cologneData.columns[0:9],axis=1)
essenData = essenData.drop(essenData.columns[0:9],axis=1)

# returns number of rented bikes per hour in essen
RentedBikesPerTimestampEss = pd.DataFrame(essenData["Zeitstempel"].value_counts())
RentedBikesPerTimestampEss["numOfRentedBikes"] = RentedBikesPerTimestampEss["Zeitstempel"]
RentedBikesPerTimestampEss.drop(columns="Zeitstempel", inplace=True)
RentedBikesPerTimestampEss["Zeitstempel"] = RentedBikesPerTimestampEss.index
RentedBikesPerTimestampEss.reset_index(drop = True, inplace=True)

# returns number of rented bikes per hour in cologne
RentedBikesPerTimestampCol = pd.DataFrame(cologneData["Zeitstempel"].value_counts())
RentedBikesPerTimestampCol["numOfRentedBikes"] = RentedBikesPerTimestampCol["Zeitstempel"]
RentedBikesPerTimestampCol.drop(columns="Zeitstempel", inplace=True)
RentedBikesPerTimestampCol["Zeitstempel"] = RentedBikesPerTimestampCol.index
RentedBikesPerTimestampCol.reset_index(drop = True, inplace=True)

#merge the important data together and drop the duplicates
cologneData = RentedBikesPerTimestampCol.merge(cologneData, left_on=['Zeitstempel'], right_on=['Zeitstempel'])
cologneData.drop_duplicates(keep="first", inplace=True)
cologneData.reset_index(drop=True, inplace=True)
essenData = RentedBikesPerTimestampEss.merge(essenData, left_on=['Zeitstempel'], right_on=['Zeitstempel'])
essenData.drop_duplicates(keep="first", inplace=True)
essenData.reset_index(drop=True, inplace=True)

So now the the given dataset looks as following:

In [25]:
essenData

,numOfRentedBikes,Zeitstempel
0,46,2019-09-30 14:00:00
1,40,2019-10-24 07:00:00
2,38,2019-09-30 15:00:00
3,34,2019-09-30 16:00:00
4,34,2019-10-24 15:00:00
...,...,...
7820,1,2019-04-28 09:00:00
7821,1,2020-01-03 10:00:00
7822,1,2019-06-02 10:00:00
7823,1,2019-09-29 08:00:00


In [26]:
cologneData

,numOfRentedBikes,Zeitstempel
0,402,2019-07-03 16:00:00
1,400,2019-06-18 14:00:00
2,400,2019-08-06 15:00:00
3,398,2019-06-04 15:00:00
4,393,2019-07-01 16:00:00
...,...,...
8680,2,2020-01-10 04:00:00
8681,2,2019-12-26 03:00:00
8682,1,2019-02-19 03:00:00
8683,1,2019-10-29 01:00:00


## Importing, cleaning and structuring the weather data

Columns description:

- Zeitstempel — hourly date + timestamp
- SDO_ID - location id of weather station 1303 essen, 2667 cologne/bonn
- isWeekend — whether the day is on a weekend 1, or not 0

- airPressure - air pressure in hpa
- airTemperature — temperature in celsius measured in a height of 2m
- cloudCoverage - cloud coverage in eighths
- relativeHumidity - relative humidity in percent
- precipationAmount - precipation amount in mm
- windVelocity — wind velocity 10 m above ground in m/s

- dayOfTheWeek - number representing the weekday

- numOfRentedBikes — number of total rentals

In [27]:
#import all the weather data listed above
airPressure = pd.read_csv("../weather_data/3_weather_data/air_pressure/data_air_pressure_hpa_hourly.csv")
airTemperature = pd.read_csv("../weather_data/3_weather_data/air_temperature/data_Temperature_air_2m_hourly.csv")
cloudCoverage = pd.read_csv("../weather_data/3_weather_data/cloud_coverage/data_Hourly_observ_cloud_coverage.csv")
precipitationAmount = pd.read_csv("../weather_data/3_weather_data/precipitation_amount/data_volume_rain_precipitation_hourly.csv")
relativeHumidity= pd.read_csv("../weather_data/3_weather_data/relative_humidity_percent/data_relative_humidity_hourly.csv")
windVelocity = pd.read_csv("../weather_data/3_weather_data/wind_velocity/data_Wind_velocity_10m_hourly.csv")

**The following code block:**  
  
1. makes a datetime-object from the column "Zeitstempel" for every weather-dataframe
2. deletes all unnecessary columns ["Produkt_Code", "Qualitaet_Niveau", "Qualitaet_Byte"]
3. divides the data into seperate dataframes for each city (essen, cologne) so that it is easier to distinguish them

In [28]:
def ts_match (ts):
    return datetime.strptime(str(ts), '%Y%m%d%H%M')

#make a datetime-object from the column "Zeitstempel" for every weather-dataframe
airTemperature["Zeitstempel"] = airTemperature["Zeitstempel"].map(ts_match)
airPressure["Zeitstempel"] = airPressure["Zeitstempel"].map(ts_match)
cloudCoverage["Zeitstempel"] = cloudCoverage["Zeitstempel"].map(ts_match)
precipitationAmount["Zeitstempel"] = precipitationAmount["Zeitstempel"].map(ts_match)
relativeHumidity["Zeitstempel"] = relativeHumidity["Zeitstempel"].map(ts_match)
windVelocity["Zeitstempel"] = windVelocity["Zeitstempel"].map(ts_match)

#delete all unnecessary columns ["Produkt_Code", "Qualitaet_Niveau", "Qualitaet_Byte"]
airPressure = airPressure.drop(columns=["Produkt_Code", "Qualitaet_Niveau", "Qualitaet_Byte"])
airTemperature = airTemperature.drop(columns=["Produkt_Code", "Qualitaet_Niveau", "Qualitaet_Byte"])
cloudCoverage = cloudCoverage.drop(columns=["Produkt_Code", "Qualitaet_Niveau", "Qualitaet_Byte"])
precipitationAmount = precipitationAmount.drop(columns=["Produkt_Code", "Qualitaet_Niveau", "Qualitaet_Byte"])
relativeHumidity = relativeHumidity.drop(columns=["Produkt_Code", "Qualitaet_Niveau", "Qualitaet_Byte"])
windVelocity = windVelocity.drop(columns=["Produkt_Code", "Qualitaet_Niveau", "Qualitaet_Byte"])

#divide the data into seperate dataframes for each city (essen, cologne)
airPressureColBo = airPressure.loc[airPressure.SDO_ID == 2667]
airPressureEss= airPressure.loc[airPressure.SDO_ID == 1303]

airTemperatureColBo = airTemperature.loc[airTemperature.SDO_ID == 2667]
airTemperatureEss= airTemperature.loc[airTemperature.SDO_ID == 1303]

cloudCoverageColBo = cloudCoverage.loc[cloudCoverage.SDO_ID == 2667]
cloudCoverageEss= cloudCoverage.loc[cloudCoverage.SDO_ID == 1303]

precipitationAmountColBo = precipitationAmount.loc[precipitationAmount.SDO_ID == 2667]
precipitationAmountEss= precipitationAmount.loc[precipitationAmount.SDO_ID == 1303]

relativeHumidityColBo = relativeHumidity.loc[relativeHumidity.SDO_ID == 2667]
relativeHumidityEss= relativeHumidity.loc[relativeHumidity.SDO_ID == 1303]

windVelocityColBo = windVelocity.loc[windVelocity.SDO_ID == 2667]
windVelocityEss= windVelocity.loc[windVelocity.SDO_ID == 1303]

**The following code block:**  
  
1. merges every weather dataframe together with each city --> fullRentalDataEssen, fullRentalDataCologne

In [29]:
# merge airPressure and airTemperature in weatherData
weatherDataEss = airPressureEss.merge(airTemperatureEss, left_on=['Zeitstempel', 'SDO_ID'], right_on=['Zeitstempel', 'SDO_ID'])

#merge cloudCoverage in weatherData - loss of some entries -> Approach: Deletion - Missing Rows
weatherDataEss = weatherDataEss.merge(cloudCoverageEss, left_on=['Zeitstempel', 'SDO_ID'], right_on=['Zeitstempel', 'SDO_ID'])

# merge windVelocity in weatherData - no loss
weatherDataEss = weatherDataEss.merge(windVelocityEss, left_on=['Zeitstempel', "SDO_ID"], right_on=['Zeitstempel', "SDO_ID"])


weatherDataEss = weatherDataEss.merge(precipitationAmountEss, left_on=['Zeitstempel', "SDO_ID"], right_on=['Zeitstempel', "SDO_ID"])
weatherDataEss = weatherDataEss.merge(relativeHumidityEss, left_on=['Zeitstempel', "SDO_ID"], right_on=['Zeitstempel', "SDO_ID"])

#final naming of the columns
weatherDataEss.columns = ['SDO_ID', 'Zeitstempel', 'airPressure', 'airTemperature', 'cloudCoverage', 'windVelocity',"precipitationAmount", "relativeHumidity"]

# cut weather data before and after the data of collected bike rental data
weatherDataEss = weatherDataEss.loc[weatherDataEss["Zeitstempel"] < "2020-01-21 00:00:00"]
weatherDataEss = weatherDataEss.loc[weatherDataEss["Zeitstempel"] >= "2019-01-20 00:00:00"]

# prepare dataframes for join
weatherDataEss = weatherDataEss.set_index('Zeitstempel')
essenData = essenData.set_index('Zeitstempel')

fullRentalDataEssen = weatherDataEss.join(essenData, how='left')
fullRentalDataEssen["numOfRentedBikes"].fillna(0, inplace=True)

fullRentalDataEssen.reset_index(inplace=True)

# cut entries during the time that no bike rental was tracked
part1_ess = fullRentalDataEssen.loc[fullRentalDataEssen["Zeitstempel"] < "2019-03-16 00:00:00"]
part2_ess = fullRentalDataEssen.loc[fullRentalDataEssen["Zeitstempel"] > "2019-03-18 23:00:00"]

fullRentalDataEssen = part1_ess.append(part2_ess)

fullRentalDataEssen = fullRentalDataEssen.drop(columns=["SDO_ID"])
# fullRentalDataEssen.reset_index(drop=True, inplace=True)
fullRentalDataEssen

,Zeitstempel,airPressure,airTemperature,cloudCoverage,windVelocity,precipitationAmount,relativeHumidity,numOfRentedBikes
0,2019-01-20 00:00:00,996.5,-2.6,0,3.2,0.0,70,2.0
1,2019-01-20 01:00:00,996.4,-2.1,0,2.6,0.0,61,0.0
2,2019-01-20 02:00:00,996.4,-2.5,0,2.7,0.0,63,0.0
3,2019-01-20 03:00:00,996.5,-3.3,0,2.0,0.0,69,3.0
4,2019-01-20 04:00:00,996.6,-4.1,1,2.4,0.0,76,1.0
...,...,...,...,...,...,...,...,...
8719,2020-01-20 19:00:00,1027.5,2.4,8,1.5,0.0,100,11.0
8720,2020-01-20 20:00:00,1027.4,1.9,8,0.4,0.0,100,8.0
8721,2020-01-20 21:00:00,1027.3,0.9,8,1.4,0.0,100,8.0
8722,2020-01-20 22:00:00,1027.2,0.5,7,1.0,0.0,100,8.0


In [30]:
# merge airPressure and airTemperature in weatherData - 9504 entries
weatherDataColBo = airPressureColBo.merge(airTemperatureColBo, left_on=['Zeitstempel', 'SDO_ID'], right_on=['Zeitstempel', 'SDO_ID'])

#merge cloudCoverage in weatherData - loss of one entrie -> Approach: Deletion - Missing Rows
weatherDataColBo = weatherDataColBo.merge(cloudCoverageColBo, left_on=['Zeitstempel', 'SDO_ID'], right_on=['Zeitstempel', 'SDO_ID'])

#merge windVelocity in weatherData - no loss
weatherDataColBo = weatherDataColBo.merge(windVelocityColBo, left_on=['Zeitstempel', "SDO_ID"], right_on=['Zeitstempel', "SDO_ID"])

weatherDataColBo = weatherDataColBo.merge(precipitationAmountColBo, left_on=['Zeitstempel', "SDO_ID"], right_on=['Zeitstempel', "SDO_ID"])
weatherDataColBo = weatherDataColBo.merge(relativeHumidityColBo, left_on=['Zeitstempel', "SDO_ID"], right_on=['Zeitstempel', "SDO_ID"])

#final naming of the columns
weatherDataColBo.columns = ['SDO_ID', 'Zeitstempel', 'airPressure', 'airTemperature', 'cloudCoverage', 'windVelocity',"precipitationAmount","relativeHumidity"]


# cut weather data before and after the data of collected bike rental data
weatherDataColBo = weatherDataColBo.loc[weatherDataColBo["Zeitstempel"] < "2020-01-21 00:00:00"]
weatherDataColBo = weatherDataColBo.loc[weatherDataColBo["Zeitstempel"] >= "2019-01-20 00:00:00"]

# prepare dataframes for join
weatherDataColBo = weatherDataColBo.set_index('Zeitstempel')
cologneData = cologneData.set_index('Zeitstempel')

fullRentalDataCologne = weatherDataColBo.join(cologneData, how='left')
fullRentalDataCologne["numOfRentedBikes"].fillna(0, inplace=True)

fullRentalDataCologne.reset_index(inplace=True)

# cut entries during the time that no bike rental was tracked
part1_col = fullRentalDataCologne.loc[fullRentalDataCologne["Zeitstempel"] < "2019-03-16 00:00:00"]
part2_col = fullRentalDataCologne.loc[fullRentalDataCologne["Zeitstempel"] > "2019-03-18 23:00:00"]

fullRentalDataCologne = part1_col.append(part2_col)

fullRentalDataCologne = fullRentalDataCologne.drop(columns=["SDO_ID"])
# fullRentalDataEssen.reset_index(drop=True, inplace=True)
fullRentalDataCologne


,Zeitstempel,airPressure,airTemperature,cloudCoverage,windVelocity,precipitationAmount,relativeHumidity,numOfRentedBikes
0,2019-01-20 00:00:00,1002.6,-3.4,1,2.6,0.0,75,61.0
1,2019-01-20 01:00:00,1002.6,-5.2,1,1.8,0.0,82,39.0
2,2019-01-20 02:00:00,1002.8,-4.7,1,1.7,0.0,80,27.0
3,2019-01-20 03:00:00,1002.9,-5.6,1,2.2,0.0,84,25.0
4,2019-01-20 04:00:00,1002.8,-4.3,1,1.8,0.0,74,10.0
...,...,...,...,...,...,...,...,...
8778,2020-01-20 19:00:00,1034.0,0.4,0,0.8,0.0,98,121.0
8779,2020-01-20 20:00:00,1033.9,-1.0,0,1.5,0.0,98,96.0
8780,2020-01-20 21:00:00,1034.0,-1.0,1,2.1,0.0,96,54.0
8781,2020-01-20 22:00:00,1033.7,0.6,1,1.6,0.0,89,68.0


## Feature Extraction

In [31]:
def isWeekend(number):
    if(number >=0 and number <= 4):
        return 0
    else:
        return 1

**The following code block adds the columns to the dataset:**  

"dayOfTheWeek": gives an integer value back for the day of the week (eg. "0" = monday)  
"hour": shows the hour (eg. "14)  
"isWeekend": 0 = weekday 1= weekend

In [32]:
# add dayoftheweek and isweekend feautures
fullRentalDataEssen["dayOfTheWeek"] = fullRentalDataEssen["Zeitstempel"].map(weekday_match)
fullRentalDataEssen["isWeekend"] = fullRentalDataEssen["dayOfTheWeek"].map(isWeekend)
fullRentalDataEssen["hour"] = fullRentalDataEssen["Zeitstempel"].map(hour_match)

In [33]:
# add dayoftheweek, isweekend and hour features
fullRentalDataCologne["dayOfTheWeek"] = fullRentalDataCologne["Zeitstempel"].map(weekday_match)
fullRentalDataCologne["isWeekend"] = fullRentalDataCologne["dayOfTheWeek"].map(isWeekend)
fullRentalDataCologne["hour"] = fullRentalDataCologne["Zeitstempel"].map(hour_match)

**The resulting dataframes looks like this:**

In [34]:
fullRentalDataCologne

,Zeitstempel,airPressure,airTemperature,cloudCoverage,windVelocity,precipitationAmount,relativeHumidity,numOfRentedBikes,dayOfTheWeek,isWeekend,hour
0,2019-01-20 00:00:00,1002.6,-3.4,1,2.6,0.0,75,61.0,6,1,0
1,2019-01-20 01:00:00,1002.6,-5.2,1,1.8,0.0,82,39.0,6,1,1
2,2019-01-20 02:00:00,1002.8,-4.7,1,1.7,0.0,80,27.0,6,1,2
3,2019-01-20 03:00:00,1002.9,-5.6,1,2.2,0.0,84,25.0,6,1,3
4,2019-01-20 04:00:00,1002.8,-4.3,1,1.8,0.0,74,10.0,6,1,4
...,...,...,...,...,...,...,...,...,...,...,...
8778,2020-01-20 19:00:00,1034.0,0.4,0,0.8,0.0,98,121.0,0,0,19
8779,2020-01-20 20:00:00,1033.9,-1.0,0,1.5,0.0,98,96.0,0,0,20
8780,2020-01-20 21:00:00,1034.0,-1.0,1,2.1,0.0,96,54.0,0,0,21
8781,2020-01-20 22:00:00,1033.7,0.6,1,1.6,0.0,89,68.0,0,0,22


In [35]:
fullRentalDataEssen

,Zeitstempel,airPressure,airTemperature,cloudCoverage,windVelocity,precipitationAmount,relativeHumidity,numOfRentedBikes,dayOfTheWeek,isWeekend,hour
0,2019-01-20 00:00:00,996.5,-2.6,0,3.2,0.0,70,2.0,6,1,0
1,2019-01-20 01:00:00,996.4,-2.1,0,2.6,0.0,61,0.0,6,1,1
2,2019-01-20 02:00:00,996.4,-2.5,0,2.7,0.0,63,0.0,6,1,2
3,2019-01-20 03:00:00,996.5,-3.3,0,2.0,0.0,69,3.0,6,1,3
4,2019-01-20 04:00:00,996.6,-4.1,1,2.4,0.0,76,1.0,6,1,4
...,...,...,...,...,...,...,...,...,...,...,...
8719,2020-01-20 19:00:00,1027.5,2.4,8,1.5,0.0,100,11.0,0,0,19
8720,2020-01-20 20:00:00,1027.4,1.9,8,0.4,0.0,100,8.0,0,0,20
8721,2020-01-20 21:00:00,1027.3,0.9,8,1.4,0.0,100,8.0,0,0,21
8722,2020-01-20 22:00:00,1027.2,0.5,7,1.0,0.0,100,8.0,0,0,22


Different amount of entries between Essen and Cologne Data is because there is not a rental in every hour of in Essen.  
Solved that problem by filling them with 0 rentals in that period.

# `Import for Task 3`

In [36]:
# already done

In [37]:
#fullRentalDataEssen.to_csv("fullRentalDataEssen.csv", index= False)

In [38]:
#fullRentalDataCologne.to_csv("fullRentalDataCologne.csv", index= False)